<a href="https://colab.research.google.com/github/SREEHARSHA04/Machine_Learning_Practice/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Downloading all the stopwords in english to remove them form our daraset

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [ ]:
## Unzipping the data file
!unzip "Fake_News.zip"

Archive:  Fake_News.zip
  inflating: fake_or_real_news.csv   


In [ ]:
news_dataset = pd.read_csv('/content/fake_or_real_news.csv')

In [ ]:
news_dataset.shape

(6335, 4)

In [ ]:
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
# Counting the number of missing values in the dataset
news_dataset.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
content       0
dtype: int64

In [ ]:
## Performing label encoding for the label column
label_encoder = LabelEncoder()   ## 0 - fake 1 - real
news_dataset['label'] = label_encoder.fit_transform(news_dataset['label'])

In [ ]:
# Separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']
print(X)
print(Y)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  \
0     Daniel Greenfield, a Shillman Journalism Fello...   
1     Google Pinterest Digg L

Stemming:

Stemming is the process of reducing a word to its Root word


In [ ]:
port_stem = PorterStemmer()

Performing the process of stemming, replacing the non alphabetic characters with space, converting all alphabetic characters to lower case and removing the stop words

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['title'].apply(stemming)

In [ ]:
print(news_dataset['title'])

0                            You Can Smell Hillary’s Fear
1       Watch The Exact Moment Paul Ryan Committed Pol...
2             Kerry to go to Paris in gesture of sympathy
3       Bernie supporters on Twitter erupt in anger ag...
4        The Battle of New York: Why This Primary Matters
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: title, Length: 6335, dtype: object


In [ ]:
#Separating the data and label
X = news_dataset['title'].values
Y = news_dataset['label'].values

In [ ]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 10025)	0.3697021492483388
  (0, 8298)	0.6428362622653288
  (0, 4285)	0.25932346019107133
  (0, 3468)	0.496138614294019
  (0, 1484)	0.3697021492483388
  (1, 9771)	0.2659708894219386
  (1, 9621)	0.20533429908741216
  (1, 9288)	0.12733666204154537
  (1, 9001)	0.10642651054603143
  (1, 8729)	0.3353619548852212
  (1, 7780)	0.27336834606331245
  (1, 7210)	0.2886914409638959
  (1, 6772)	0.25269859002770484
  (1, 6536)	0.25081525496784146
  (1, 5813)	0.3166556995288715
  (1, 3266)	0.4180253197427375
  (1, 1926)	0.38604676499215423
  (1, 770)	0.1954545168939325
  (2, 9098)	0.2739800153499639
  (2, 8843)	0.49398475717214113
  (2, 6486)	0.3475966924968782
  (2, 6225)	0.16125634950154008
  (2, 5002)	0.38571138240303604
  (2, 4549)	0.1559235749972116
  (2, 3955)	0.34202422122927323
  :	:
  (6332, 6259)	0.4664328071894896
  (6332, 6225)	0.16551737349827622
  (6332, 4624)	0.37413697990466194
  (6332, 677)	0.25650820420765597
  (6332, 598)	0.33057775047306454
  (6333, 7955)	0.33787437103176404
 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=45)

In [ ]:
## Training the Model: Logistic Regression
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.92008681925809


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.8200473559589582
